# BasicTick: Create DB

## Steps
1. Untar hdb.tar.gz for the hdb data
2. Upload hdb to staging S3 bucket
3. Create database


## Setup
imports, environmentId, accountId

In [1]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env2 import *


In [2]:
# Source data directory
SOURCE_DATA_DIR="hdb"

# S3 bucket for external data and code
S3_DEST=f"s3://{S3_BUCKET}/data/{SOURCE_DATA_DIR}/"

# Managed KX Database and Cluster names to create
DB_NAME="basictickdb"

In [3]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


## 1. Untar hdb.tar.gz
hdb database will be found in hdb directory

In [4]:
!tar -xf hdb.tar.gz

In [5]:
!ls -la hdb

total 52
drwxr-xr-x. 12 ec2-user ec2-user 16384 Apr 24 23:17 .
drwxr-xr-x.  7 ec2-user ec2-user 16384 May 26 16:26 ..
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.14
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.15
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.16
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.17
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.18
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.19
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.20
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.21
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.22
drwxr-xr-x.  3 ec2-user ec2-user    21 Apr 24 23:17 2023.04.23
-rw-r--r--.  1 ec2-user ec2-user 16392 Apr 24 23:17 sym


# 2. Upload hdb data
using aws cli, copy hdb to staging bucket

In [6]:
if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

aws s3 sync  --exclude .DS_Store {SOURCE_DATA_DIR} {S3_DEST}
aws s3 ls {S3_DEST}
"""
else:
    cp = f"""
aws s3 sync  --exclude .DS_Store {SOURCE_DATA_DIR} {S3_DEST}
aws s3 ls {S3_DEST}
"""
    
# execute the S3 copy
os.system(cp)

                           PRE 2023.01.29/
                           PRE 2023.01.30/
                           PRE 2023.01.31/
                           PRE 2023.02.01/
                           PRE 2023.02.02/
                           PRE 2023.02.03/
                           PRE 2023.02.04/
                           PRE 2023.02.05/
                           PRE 2023.02.06/
                           PRE 2023.02.07/
                           PRE 2023.04.14/
                           PRE 2023.04.15/
                           PRE 2023.04.16/
                           PRE 2023.04.17/
                           PRE 2023.04.18/
                           PRE 2023.04.19/
                           PRE 2023.04.20/
                           PRE 2023.04.21/
                           PRE 2023.04.22/
                           PRE 2023.04.23/
2023-04-27 17:42:55      16392 sym


0

## 3. Create database

In [7]:
# assume it exists
create_db=False

try:
    resp = client.get_kx_database(environmentId=ENV_ID, databaseName=DB_NAME)
    resp.pop('ResponseMetadata', None)
except:
    # does not exist, will create
    create_db=True

if create_db:
    print(f"CREATING Database: {DB_NAME}")
    resp = client.create_kx_database(environmentId=ENV_ID, databaseName=DB_NAME, description="Welcome kdb database")
    resp.pop('ResponseMetadata', None)

    print(f"CREATED Database: {DB_NAME}")

print(json.dumps(resp,sort_keys=True,indent=4,default=str))

CREATING Database: basictickdb
CREATED Database: basictickdb
{
    "createdTimestamp": "2023-05-30 23:21:59.431000+00:00",
    "databaseArn": "arn:aws:finspace:us-east-1:612841383594:kxEnvironment/itcdoumzc5cixt5vh4t6dp/kxDatabase/basictickdb",
    "databaseName": "basictickdb",
    "description": "Welcome kdb database",
    "environmentId": "itcdoumzc5cixt5vh4t6dp",
    "lastModifiedTimestamp": "2023-05-30 23:21:59.431000+00:00"
}


## 4. Add HDB data to database

In [8]:
changes=[]

for f in os.listdir("hdb"):
    if os.path.isdir(f"hdb/{f}"):
        changes.append( { 'changeType': 'PUT', 's3Path': f"{S3_DEST}{f}/", 'dbPath': f"/{f}/" } )
    else:
        changes.append( { 'changeType': 'PUT', 's3Path': f"{S3_DEST}{f}", 'dbPath': f"/" } )
        
resp = client.create_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, 
    changeRequests=changes)

resp.pop('ResponseMetadata', None)
changeset_id = resp['changesetId']

print("Changeset...")
print(json.dumps(resp,sort_keys=True,indent=4,default=str))

Changeset...
{
    "changeRequests": [
        {
            "changeType": "PUT",
            "dbPath": "/2023.04.23/",
            "s3Path": "s3://kdb-demo-612841383594/data/hdb/2023.04.23/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2023.04.15/",
            "s3Path": "s3://kdb-demo-612841383594/data/hdb/2023.04.15/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2023.04.14/",
            "s3Path": "s3://kdb-demo-612841383594/data/hdb/2023.04.14/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2023.04.22/",
            "s3Path": "s3://kdb-demo-612841383594/data/hdb/2023.04.22/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2023.04.18/",
            "s3Path": "s3://kdb-demo-612841383594/data/hdb/2023.04.18/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2023.04.20/",
            "s3Path": "s3://kdb-demo-612841383594/data/hdb/2

In [9]:
wait_for_changeset_status(client, ENV_ID, DB_NAME, changeset_id, show_wait=True)

Status is IN_PROGRESS, total wait 0:00:00, waiting 10 sec ...
Status is IN_PROGRESS, total wait 0:00:10, waiting 10 sec ...
Status is IN_PROGRESS, total wait 0:00:20, waiting 10 sec ...


{'changesetId': 'ZsQ3eXNny4UaexiXtmEkZg',
 'databaseName': 'basictickdb',
 'environmentId': 'itcdoumzc5cixt5vh4t6dp',
 'changeRequests': [{'changeType': 'PUT',
   's3Path': 's3://kdb-demo-612841383594/data/hdb/2023.04.23/',
   'dbPath': '/2023.04.23/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-612841383594/data/hdb/2023.04.15/',
   'dbPath': '/2023.04.15/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-612841383594/data/hdb/2023.04.14/',
   'dbPath': '/2023.04.14/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-612841383594/data/hdb/2023.04.22/',
   'dbPath': '/2023.04.22/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-612841383594/data/hdb/2023.04.18/',
   'dbPath': '/2023.04.18/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-612841383594/data/hdb/2023.04.20/',
   'dbPath': '/2023.04.20/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-612841383594/data/hdb/2023.04.16/',
   'dbPath': '/2023.04.16/'},
  {'changeType': 'PUT',
   's3Path': '

In [10]:
note_str = ""

c_set_list = list_kx_changesets(client, ENV_ID, DB_NAME)

if len(c_set_list) == 0:
    note_str = "<<Could not get changesets>>"
    
print(100*"=")
print(f"Database: {DB_NAME}, Changesets: {len(c_set_list)} {note_str}")
print(100*"=")

# sort by create time
c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

for c in c_set_list:
    c_set_id = c['changesetId']
    print(f"  Changeset: {c_set_id}: Created: {c['createdTimestamp']} ({c['status']})")
    c_rqs = client.get_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, changesetId=c_set_id)['changeRequests']

    chs_pdf = pd.DataFrame.from_dict(c_rqs).style.hide(axis='index')
    display(chs_pdf)

Database: basictickdb, Changesets: 1 
  Changeset: ZsQ3eXNny4UaexiXtmEkZg: Created: 2023-05-30 23:22:00.273000+00:00 (COMPLETED)


changeType,s3Path,dbPath
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.23/,/2023.04.23/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.15/,/2023.04.15/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.14/,/2023.04.14/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.22/,/2023.04.22/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.18/,/2023.04.18/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.20/,/2023.04.20/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.16/,/2023.04.16/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.17/,/2023.04.17/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.21/,/2023.04.21/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.19/,/2023.04.19/


In [11]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2023-05-30 23:22:33.172750
